<a href="https://colab.research.google.com/github/ratmcu/wiki_ner/blob/master/cntry_lst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import urllib
from nltk.tokenize import sent_tokenize
import nltk
import re

class hashabledict(dict):
    def __hash__(self):
        return hash(tuple(sorted(self.items())))

In [0]:
# url = 'https://en.wikipedia.org/wiki/List_of_sovereign_states'
# quote_page = url
# page = urllib.request.urlopen(quote_page)
# soup = BeautifulSoup(page, 'html.parser')
# table_pane = soup.find('div', attrs={'class': 'mw-parser-output'})
# table_entry_list = table_pane.find('tbody').find_all('tr')
# country_url = 'https://en.wikipedia.org' + table_entry_list[5].a.get('href')
# print(country_url)
# quote_page = url
# page = urllib.request.urlopen(quote_page)
# soup = BeautifulSoup(page, 'html.parser')

In [0]:
def get_country_urls(lst_url):
  cquote_page = lst_url
  cpage = urllib.request.urlopen(cquote_page)
  csoup = BeautifulSoup(cpage, 'html.parser')
#   side_pane = soup.find('table', attrs={'class': 'infobox vcard'})
  table_pane = csoup.find('div', attrs={'class': 'mw-parser-output'})
#   table_entry_list = table_pane.find('tbody').find_all('tr')
  table_entry_list = csoup.find('tbody').find_all('tr')
  contry_url_list = []
  for table_entry in table_entry_list:
    if table_entry.a :
      if table_entry.a.get('href')[0] != '#':
#         print(table_entry.a.get('title'))
        contry_url_list.append(hashabledict({'country':table_entry.a.get('title'),
                                'url':'https://en.wikipedia.org'+table_entry.a.get('href')}))
#         print(table_entry.a.text, 'https://en.wikipedia.org'+table_entry.a.get('href'))
  return contry_url_list

def get_president_link(prs_url):
  cquote_page = prs_url
  cpage = urllib.request.urlopen(cquote_page)
  csoup = BeautifulSoup(cpage, 'html.parser')
#   table_pane = csoup.find('div', attrs={'class': 'mw-parser-output'})
#   table_entry_list = table_pane.find('tbody').find_all('tr')
  table_entry_list = csoup.find('tbody').find_all('tr')
  contry_url_list = []
  for table_entry in table_entry_list:
#     if table_entry.a :
#       if table_entry.a.text == 'President':
#         print(table_entry.a.text, 'https://en.wikipedia.org'+table_entry.a.get('href'))
    if table_entry.a and table_entry.a.text == 'President':
#     if table_entry.a and table_entry.a.text == 'Prime Minister':
#       print(table_entry)
      for child in table_entry.children:
#         if child.text == 'President':
        if child.a.text!='President':
#           print('https://en.wikipedia.org'+child.a.get('href'))
#           print(child.a.get('title'))
          return hashabledict({'name':child.a.get('title'), 'url':'https://en.wikipedia.org'+child.a.get('href')})
 
def get_preceded_link(prs_url):
  cquote_page = prs_url['url']
  try:
    cpage = urllib.request.urlopen(cquote_page)
  except:
    return None
  csoup = BeautifulSoup(cpage, 'html.parser')
#   table_pane = csoup.find('div', attrs={'class': 'mw-parser-output'})
#   table_entry_list = table_pane.find('tbody').find_all('tr')
  table_entry_list = csoup.find_all('tr')
#   print(csoup.find('tbody'))
  for table_entry in table_entry_list:
#     if table_entry :
#     print(table_entry)
#       if table_entry.a.text == 'President':
#         print(table_entry.a.text, 'https://en.wikipedia.org'+table_entry.a.get('href'))
    if table_entry.th and table_entry.th.text == 'Preceded by':
#     if table_entry.a and table_entry.a.text == 'Prime Minister':
      try:
#         print('https://en.wikipedia.org'+table_entry.a.get('href'))
#         return 'https://en.wikipedia.org'+table_entry.a.get('href')
        return hashabledict({'name':table_entry.a.get('title'), 'url':'https://en.wikipedia.org'+table_entry.a.get('href')})
      except:
        return None

def stack_presidents(purl):
  p_list = []
#   for purl in purl_list:
  p_set = set()
#   p_set.add(purl)
  if purl:
    p_list.append(purl)
#     print('1')
    pre_url = get_preceded_link(purl)
    while pre_url and pre_url not in p_set:
#       print('2')
      p_set.add(pre_url)
      pre_url = get_preceded_link(pre_url)
  p_list.extend(p_set)  
  return p_list
    
    

In [0]:
country_list = get_country_urls('https://en.wikipedia.org/wiki/List_of_sovereign_states')
# get_president_link('https://en.wikipedia.org/wiki/Albania')
# lists = [get_president_link(url) for url in country_list]
# cntry_pres_dict = {'country':{}}
# country_list
# lists = [ pres if pres = get_president_link(url) for url in country_list]
# print(get_preceded_link(get_preceded_link(get_preceded_link(get_preceded_link('https://en.wikipedia.org/wiki/Ilir_Meta')))))
# get_preceded_link('https://en.wikipedia.org/wiki/Bamir_Topi')
# get_preceded_link('https://en.wikipedia.org/wiki/Alfred_Moisiu')

In [0]:
# cntry_pres_dict = {}
# cntry_pres_dict.update({'Sri Lanka':
#                                     {'url':'',
#                                      'presidents':[]}}) #{'name':''    <-- this will be an element in the list of the presidents
#                                                         # 'url' :''}
for i in range(len(country_list)):
  country_list[i].update({'presidents':[]})
  country_list[i]['presidents'].extend(stack_presidents(get_president_link(country_list[i]['url'])))
# country_list[3]
#   country_list[3]
                                     

In [0]:
import pickle
# pres_tot_lst = stack_presidents(lists)
with open('president_list.pkl', 'wb') as f:
    pickle.dump(country_list, f)

In [0]:
with open('president_list.pkl', 'rb') as f:
    prsdnt_list = pickle.load(f)
    
len(prsdnt_list)

for cntry in prsdnt_list:
  print(cntry['country'])
#   os.makedirs('politicians' + '/' + )
#   for prsdnt in cntry['presidents']:

United Nations System
Member states of the United Nations
Afghanistan
Albania
Algeria
Andorra
Angola
Antigua and Barbuda
Argentina
Armenia
Australia
Austria
Azerbaijan
The Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Cape Verde
Central African Republic
Chad
Chile
China
Colombia
Comoros
Democratic Republic of the Congo
Republic of the Congo
Costa Rica
Croatia
Cuba
Cyprus
Czech Republic
Denmark
Djibouti
Dominica
Dominican Republic
East Timor
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Fiji
Finland
France
Gabon
The Gambia
Georgia (country)
Germany
Ghana
Greece
Grenada
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Republic of Ireland
Israel
Italy
Ivory Coast
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kiribati
North Korea
South Korea
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Lesotho
L

In [0]:
len(pres_tot_lst)
with open('/home/presidents.txt', 'w') as f:
    for item in pres_tot_lst:
        f.write("%s\n" % item)

NameError: ignored

In [0]:
pres_tot_lst

In [0]:
class hashabledict(dict):
    def __hash__(self):
        return hash(tuple(sorted(self.items())))
      
hd1 = hashabledict({'one':1,'two':2})
hd2 = hashabledict({'one':1,'tw3':2})
p_set = set([])
p_set.add(hd1)
hd2 in p_set